In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import os
from time import time

In [ ]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

In [ ]:
from GetMassData import *
from ModelData import *

In [ ]:
import json

In [ ]:
#overview of the data
#dataDirectory = os.path.join(os.getcwd(),'data')
dataDirectory = os.path.join(r"E:\TopoptGAfileSaves","Mass minimization","Agents")
DATA_FILE_PATH = os.path.join(dataDirectory,'100_50')

dir_list = os.listdir(DATA_FILE_PATH)
max_data_points = len(dir_list)
print("Number of data points: {}".format(len(dir_list)))
print(dir_list[0])

In [ ]:
dataDirectory = os.path.join("E:\TopoptGAfileSaves","Mass minimization","JustMirrored","Agents","100_50")
path = dataDirectory#os.path.join(dataDirectory,'100_50')
AgentsToGrab = os.listdir(path)
numAgents = len(AgentsToGrab)
print("There are {} files to explore.".format(numAgents))

fx_array = []
fy_array = []
Youngs_array = []
compliance_array = []
stress_array = []
for i,agent in enumerate(AgentsToGrab):
    print("{:.1f}%\t".format(100*(i/numAgents)),end='\r')
    agentFiles = os.listdir(os.path.join(path,agent))
    for fileName in agentFiles:
        if('loadConditions' in fileName):
            loadConditions = np.load(os.path.join(path,agent,fileName))
            #print('loadCondtions Exist')
            formated = unpackLoadConditions(loadConditions)
            circles = formated[0]
            radii = formated[1]
            forces = formated[2]
            nelx, nely = formated[3], formated[4]
            Youngs, C_max, S_max = formated[5], formated[6], formated[7]
            fx_array.append(forces[0][0])
            fx_array.append(forces[0][1])
            fx_array.append(forces[0][2])
            fy_array.append(forces[1][0])
            fy_array.append(forces[1][1])
            fy_array.append(forces[1][2])
            Youngs_array.append(Youngs)
            compliance_array.append(C_max)
            stress_array.append(S_max)
            break
            
        


In [ ]:
print(np.mean(fx_array))
print(np.mean(fy_array))
print(np.mean(Youngs_array))
print(np.mean(compliance_array))
print(np.mean(stress_array))

In [ ]:
def formatStats(array):
    print("min:",np.min(array))
    print("mean:",np.mean(array))
    print("max:",np.max(array))

    print("Normalizing")
    # Forces
    # ar2 = np.array(array) 
    # ar2_mean = np.mean(np.abs(ar2))
    # ar2 = ar2 / ar2_mean
    ar2 = np.array(array) 
    ar2_mean = np.mean(ar2)
    ar2 = ar2 / ar2_mean
    print("min:",np.min(ar2))
    print("mean:",np.mean(ar2))
    print("max:",np.max(ar2))

    print(ar2_mean)

In [ ]:
formatStats(stress_array)

The sequence class will hold each problem statement as a sequence of all the iterations.
It will hold the load conditions as well as each iteration.
<ol>
<li>When training the model, some fraction of the sequences will be called and the model will train on them.</li>
<li>The model will then be given some other fraction of seqences to predict.</li>
<li>These predictions will be maped to their original inputs and will be re-outputed as new problem statment iterations.</li>
<li>The new(Model predicted) datapoints can then be inputed back into the model for training.</li>
</ol>

In [ ]:
FORCE_NORMILIZATION_FACTOR = 7000
YOUNGS_MODULUS_NORMILIZATION_FACTOR = 238000000000
COMPLIANCE_MAX_NORMILIZATION_FACTOR = 0.03
STRESS_MAX_NORMILIZATION_FACTOR = 15000000

class TopOptSequence:
    def __init__(self,ID,formatted,x_array,numIterations,converged):
        self.ID = ID
        self.loadCondtions = formatted
        self.xPhys_array = x_array
        self.numIterations = numIterations
        self.nelx = self.loadCondtions[3]
        self.nely = self.loadCondtions[4]
        self.converged = converged
    
    def formatLoadCondtions(self):
        circles = self.loadCondtions[0]
        radii = self.loadCondtions[1]
        forces = self.loadCondtions[2]
        nelx, nely = self.loadCondtions[3], self.loadCondtions[4]
        Youngs, C_max, S_max = self.loadCondtions[5], self.loadCondtions[6], self.loadCondtions[7]

        x = np.linspace(0,2,nelx+1)
        y = np.linspace(0,1,nely+1)
        X,Y = np.meshgrid(x,y)

        def dist(num):
            return np.sqrt((X-circles[0][num])**2 + (Y-circles[1][num])**2) - radii[num]

        circleImage = np.minimum(dist(0),np.minimum(dist(1),dist(2)))
        circleImage = np.where(circleImage >= 0, 0,1)

        circleImage = np.reshape(circleImage.T,(nelx+1,nely+1,1))

        res = min(nelx,nely)

        forceImageX = np.zeros((nelx+1,nely+1,1))
        forceImageY = np.zeros((nelx+1,nely+1,1))
        for i in range(3):
            fx = forces[0][i] / FORCE_NORMILIZATION_FACTOR
            fy = forces[1][i] / FORCE_NORMILIZATION_FACTOR
            x_coord = int(circles[0][i] * res)
            y_coord = int(circles[1][i] * res)
            forceImageX[x_coord,y_coord,0] = fx
            forceImageY[x_coord,y_coord,0] = fy

            
        #print("Y.shape:",Y.shape)

        Y_image = (Youngs / YOUNGS_MODULUS_NORMILIZATION_FACTOR )*np.ones((nelx+1,nely+1,1))
        c_max_image = (C_max / COMPLIANCE_MAX_NORMILIZATION_FACTOR )*np.ones((nelx+1,nely+1,1))
        s_max_image = (S_max / STRESS_MAX_NORMILIZATION_FACTOR )*np.ones((nelx+1,nely+1,1))

        # print("circleImage.shape:",circleImage.shape)
        # print("forceImageX.shape:",forceImageX.shape)
        # print("forceImageY.shape:",forceImageY.shape)
        # print("Y_image.shape:",Y_image.shape)
        # print("c_max_image.shape:",c_max_image.shape)
        # print("s_max_image.shape:",s_max_image.shape)

        loadCondtionsImage = np.concatenate([circleImage,forceImageX,forceImageY,Y_image,c_max_image,s_max_image],axis=2)
        return loadCondtionsImage

    def dispenceFirstIteration(self,iterationdepth:int=5,step:int=5):
        StartingBlock = np.reshape(self.xPhys_array[0],(self.nelx+1,self.nely+1,1),order='F')
        outputParts = []
        for i in range(iterationdepth):
            
            jumpIndex = min(self.numIterations-1,step*(i+1))
            outputParts.append(np.reshape(self.xPhys_array[jumpIndex],(self.nelx+1,self.nely+1,1),order='F'))

        
        formattedImage = self.formatLoadCondtions()

        return StartingBlock,formattedImage,outputParts

    def dispenceIteration(self,iterationNumber,iterationdepth:int=5,step:int=5):
        iterationNumber = min(iterationNumber,self.numIterations-1)
        StartingBlock = np.reshape(self.xPhys_array[iterationNumber],(self.nelx+1,self.nely+1,1),order='F')
        outputParts = []
        for i in range(iterationdepth):
            
            jumpIndex = min(self.numIterations-1,iterationNumber + step*(i+1))
            outputParts.append(np.reshape(self.xPhys_array[jumpIndex],(self.nelx+1,self.nely+1,1),order='F'))

        
        formattedImage = self.formatLoadCondtions()

        return StartingBlock,formattedImage,outputParts




In [ ]:
def buildDataSet(dataPointsToGrab:int):

    # Constants of interest
    # DATA_FILE_PATH = path to agent files
    # dir_List = all agent files
    # max_data_points = total number of datapoints

    dataPointsToGrab = min(dataPointsToGrab,max_data_points)

    #randomize the data grabed so that the first thee datapoints aren't always in the data.
    indexList = np.arange(max_data_points,dtype='int32')
    np.random.shuffle(indexList)
    nonConvergedCounter = 0

    sequenceData = []
    print("Retreiving {} Datapoints.".format(dataPointsToGrab))

    for i in range(dataPointsToGrab):
        print("{:.2f}%\t\t".format((100*(i/dataPointsToGrab))),end='\r')
        try:
            #join the data file path to a random sorted member within the data directory
            pathToAgent = os.path.join(DATA_FILE_PATH,dir_list[indexList[i]])
            formated,x_array,derivatives_array,objectives_array,markName = getData(pathToAgent)
            
        except:
            #if an exception occurs list it and move forward
            print("Exception Occured at file '{}'.".format(os.path.join(DATA_FILE_PATH,dir_list[indexList[i]])))
            continue
        else:
            cvrg = True
            if('NotConverged' in markName):
                print("file {} has not converged.".format(dir_list[indexList[i]]))
                nonConvergedCounter += 1
                cvrg = False
            else:
                #if no error occured append that data to the data list
                sequenceData.append(TopOptSequence(i,formated,x_array,len(x_array),cvrg))

    print("100%\t\t")
    print(f"Out of {dataPointsToGrab} data points gathered, {100*(nonConvergedCounter/dataPointsToGrab)}% had not converged for a total of {nonConvergedCounter}")
    return sequenceData
        


In [ ]:
Data = buildDataSet(30)
print(len(Data))

In [ ]:
loadCondtionsImage = Data[0].formatLoadCondtions()

In [ ]:
#Test Train Split
"""
By performing the test train split we can get a training data set and a testing dataset to get the metrics for out model
By performing the split a second time we can get a validataion dataset that the model will never see that we can use to get out own accuracy score out of
"""
Data_train, Data_test = train_test_split(Data, test_size=0.5)
print("Train: {}".format(len(Data_train)))
print("\nTest: {}".format(len(Data_test)))

<h1>Model Information</h1>

Below are the models that will be used to attempt to learn the dataset

In [ ]:
#setUp modelSaving

def getModel():
    modelNum = 9
    model = Model_m9()
    fileSaveName = "Model_m{}".format(modelNum)
    
    

    modelPath = os.path.join(os.getcwd(),'ModelSave',fileSaveName)
    
    cp_callback = keras.callbacks.ModelCheckpoint(filepath=os.path.join(modelPath,fileSaveName),
                                                     save_weights_only=True,
                                                     verbose=1)
    if(os.path.isdir(modelPath)):
        try:
            
            model.load_weights(os.path.join(modelPath,fileSaveName))
        except:
            print("Model weights could not be loaded.")
        else:
            print("Model weights Loaded")
    else:
        os.mkdir(modelPath)
        print("Model path created")

    
    
    model.compile(  optimizer='Adam',
                    loss= keras.losses.BinaryCrossentropy())
    return model,cp_callback

In [ ]:
currentModelNumber = 9 #change this one
model,callBack = getModel()
print()
model.model.summary()

In [ ]:
def plotHistory(hist):
    fig,ax = plt.subplots(1,1)
    ax.set_title("")
    keys = hist.history.keys()
    y = np.arange(1,hist.params['epochs']+1)
    for key in keys:
        if('output' in key):
            minVal = min(hist.history[key])
            meanVal = np.mean(hist.history[key])
            maxVal = max(hist.history[key])
            if(minVal != maxVal):
                print("{}:\n\tmin:{}\n\tmean:{}\n\tmax:{}".format(key,minVal,meanVal,maxVal))
                #ax.plot(y,hist.history[key],linewidth=0.5,label=key)
        else:
            ax.plot(y,hist.history[key],label=key)
            minVal = min(hist.history[key])
            meanVal = np.mean(hist.history[key])
            maxVal = max(hist.history[key])
            print("{}:\n\tmin:{}\n\tmean:{}\n\tmax:{}".format(key,minVal,meanVal,maxVal))
    
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    plt.show()

def plotHistory_lite(hist):
    fig,ax = plt.subplots(1,1)
    ax.set_title("")
    keys = hist.keys()
    for key in keys:
        if('output' in key):
            minVal = min(hist[key])
            meanVal = np.mean(hist[key])
            maxVal = max(hist[key])
            if(minVal != maxVal):
                print("{}:\n\tmin:{}\n\tmean:{}\n\tmax:{}".format(key,minVal,meanVal,maxVal))
                #ax.plot(y,hist.history[key],linewidth=0.5,label=key)
        else:
            y = np.arange(1,len(hist[key])+1)
            ax.plot(y,hist[key],label=key)
            minVal = min(hist[key])
            meanVal = np.mean(hist[key])
            maxVal = max(hist[key])
            print("{}:\n\tmin:{}\n\tmean:{}\n\tmax:{}".format(key,minVal,meanVal,maxVal))
    
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    plt.show()

In [ ]:
def preTrainModelOverFirstIteration(iterationJump:int=5):
    def createDataset(data):
        loadCondtions = []
        parts = []
        outputs = []
        for i in range(len(data)):
            StartingBlock,formattedImage,outputParts = data[i].dispenceFirstIteration(5,iterationJump)
            loadCondtions.append(formattedImage)
            parts.append(StartingBlock)
            outputArrays = []
            for outputBlock in outputParts:
                outputArrays.append(outputBlock)
            outputs.append(outputArrays)
        
        loadCondtions = np.array(loadCondtions)
        parts = np.array(parts)
        outputs = np.array(outputs)
        return loadCondtions,parts,outputs
    
    
    
    format_array,x_array,outputs_array = createDataset(Data_train)

    x1 = outputs_array[:,0,:,:,:]
    x2 = outputs_array[:,1,:,:,:]
    x3 = outputs_array[:,2,:,:,:]
    x4 = outputs_array[:,3,:,:,:]
    x5 = outputs_array[:,4,:,:,:]

    print("format_array.shape:",format_array.shape)
    print("x_array.shape:",x_array.shape)
    print("outputs_array.shape:",outputs_array.shape)
    print("x1.shape:",x1.shape)
    print("x5.shape:",x5.shape)
    numEpochs = 5
    BatchSize = 32 # default tensorflow batchsize
    numBatches = len(x_array) // BatchSize
    BatchesPerEpoch = numBatches// numEpochs
    print("Pretraining model over {} epochs.\n\tnumSamples: {}\n\tnumBatches: {}\n\tBatches per Epoch:{}\n".format(numEpochs,len(x_array),numBatches,BatchesPerEpoch))
    
    history1 = model.fit(
        x={'x':x_array,'loadConditions':format_array},
        y=(x1,x2,x3,x4,x5),
        validation_split = 0.1,
        epochs=numEpochs,
        shuffle=True,
        steps_per_epoch = BatchesPerEpoch,
        callbacks = [callBack])

    return history1

    

In [ ]:
h1 = preTrainModelOverFirstIteration(5)

In [ ]:
plotHistory(h1)

In [ ]:
def TrainModel(iterationJump:int=5):
    def createDataset(data):
        loadCondtions = []
        parts = []
        outputs = []
        for i in range(len(data)):
            if(data[i].converged):
                for j in range(data[i].numIterations):
                    StartingBlock,formattedImage,outputParts = data[i].dispenceIteration(j,5,iterationJump)
                    loadCondtions.append(formattedImage)
                    parts.append(StartingBlock)
                    outputArrays = []
                    for outputBlock in outputParts:
                        outputArrays.append(outputBlock)
                    outputs.append(outputArrays)
        
        loadCondtions = np.array(loadCondtions)
        parts = np.array(parts)
        outputs = np.array(outputs)
        return loadCondtions,parts,outputs
    
    
    
    format_array,x_array,outputs_array = createDataset(Data_train)

    x1 = outputs_array[:,0,:,:,:]
    x2 = outputs_array[:,1,:,:,:]
    x3 = outputs_array[:,2,:,:,:]
    x4 = outputs_array[:,3,:,:,:]
    x5 = outputs_array[:,4,:,:,:]

    print("format_array.shape:",format_array.shape)
    print("x_array.shape:",x_array.shape)
    print("outputs_array.shape:",outputs_array.shape)
    print("x1.shape:",x1.shape)
    print("x5.shape:",x5.shape)
    numEpochs = 5
    BatchSize = 32 # default tensorflow batchsize
    numBatches = len(x_array) // BatchSize
    BatchesPerEpoch = numBatches// numEpochs
    print("Pretraining model over {} epochs.\n\tnumSamples: {}\n\tnumBatches: {}\n\tBatches per Epoch:{}\n".format(numEpochs,len(x_array),numBatches,BatchesPerEpoch))
    
    history1 = model.fit(
        x={'x':x_array,'loadConditions':format_array},
        y=(x1,x2,x3,x4,x5),
        validation_split = 0.1,
        epochs=numEpochs,
        shuffle=True,
        steps_per_epoch = BatchesPerEpoch,
        callbacks = [callBack])

    return history1


In [ ]:
h3 = TrainModel(5)

In [ ]:
h2 = TrainModel(5)

In [ ]:
h1 = json.load(open(r"C:\Users\Nate\Documents\GitHub\SOundstuff\Top-Op\trainHistory_0",'r'))

In [ ]:
plotHistory_lite(h1)

In [ ]:
def EditImage(image):

    #noise = np.random.normal(loc=0,scale=.05,size=image.shape)
    #return image + noise

    return np.flip(image,axis=0)

In [ ]:
def DrawModelIteratins(num):
    sequenceToStart:TopOptSequence = Data_test[num]
    numImages = sequenceToStart.numIterations
    imagesToShow = 5
    #print(numImages)
    fig,ax = plt.subplots(2,imagesToShow)

    nelx = 100
    nely = 50
    StartingBlock,formattedImage,_ = sequenceToStart.dispenceFirstIteration(1,1)

    formattedImage = np.array(formattedImage)
    ImageToPredict = np.array(StartingBlock)
    PredictedImages = [ImageToPredict]

    start = time()
    for i in range(numImages):
        
        output = model.predict({'x':ImageToPredict,'loadConditions':formattedImage},verbose = 0)
        ImageToPredict = output#[0]
        PredictedImages.append(ImageToPredict)
    end = time()
    print("{} iterations took {:.2f} seconds or about {:.5f} seconds per iteration.".format(numImages,end-start,(end-start)/numImages))
    imagesToJump = numImages // imagesToShow

    for i in range(0,imagesToShow-1):
        ax[0,i].imshow(np.reshape(sequenceToStart.xPhys_array[:,:,i*imagesToJump],(nelx,nely)).T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
        ax[0,i].get_xaxis().set_visible(False)
        ax[0,i].get_yaxis().set_visible(False)

        #ax[1,i].set_title("Pred")#:{}".format(finalBit(Y_score_finished[rnd[i]])))
        ax[1,i].imshow(np.reshape(PredictedImages[i*imagesToJump],(nelx,nely)).T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
        ax[1,i].get_xaxis().set_visible(False)
        ax[1,i].get_yaxis().set_visible(False)
    ax[0,-1].imshow(np.reshape(sequenceToStart.xPhys_array[:,:,-1],(nelx,nely)).T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
    ax[0,-1].get_xaxis().set_visible(False)
    ax[0,-1].get_yaxis().set_visible(False)

    #ax[1,i].set_title("Pred")#:{}".format(finalBit(Y_score_finished[rnd[i]])))
    ax[1,-1].imshow(np.reshape(PredictedImages[-1],(nelx,nely)).T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
    ax[1,-1].get_xaxis().set_visible(False)
    ax[1,-1].get_yaxis().set_visible(False)

    ax[0,0].set_title("True")
    ax[1,0].set_title("Pred")
    plt.show()


In [ ]:
n = 0#np.random.randint(0,len(Data_score)-1)
print(n)
DrawModelIteratins(n)

In [ ]:
#n = np.random.randint(0,len(Data_score)-1)
n = np.arange(len(Data_score),dtype='int32')
np.random.shuffle(n)
print(len(n))
for i in range(10):
    DrawModelIteratins(n[i])

In [ ]:
from PIL import Image
import io

In [ ]:
def plotOverIterations(index,data):
    sequenceToStart = data[index]
    numImages = 20#sequenceToStart.numIterations
    imagesToShow = numImages
    #print(numImages)
    nelx = 101
    nely = 51
    StartingBlock,formattedImage,_ = sequenceToStart.dispenceFirstIteration(1,1)

    formattedImage = np.reshape(formattedImage,(1,nelx,nely,6))
    ImageToPredict = np.reshape(StartingBlock,(1,nelx,nely,1))
    PredictedImages = [ImageToPredict]

    start = time()
    for i in range(numImages):
        
        output = m2.predict({'x':ImageToPredict,'loadConditions':formattedImage},verbose = 0)
        ImageToPredict = output#[0]
        PredictedImages.append(ImageToPredict)
    end = time()
    print("{} iterations took {:.2f} seconds or about {:.5f} seconds per iteration.".format(numImages,end-start,(end-start)/numImages))
    imagesToJump = 1
    imageArray = []

    for i,image in enumerate(PredictedImages):
        fig,ax = plt.subplots(1,1)
        
        if(i == 0):
            ax.set_title("Iteration: {}".format(i))
        else:
            im1 = np.reshape(PredictedImages[i],(nelx*nely))
            im2 = np.reshape(PredictedImages[i-1],(nelx*nely))
            ax.set_title("Iteration: {}, Change: {:.5f}".format(i,np.linalg.norm(im1-im2,ord=np.inf)))
        ax.imshow(np.reshape(image,(nelx,nely)).T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        #img_buf = io.BytesIO()
        #plt.savefig(img_buf, format='png')

        #im = Image.open(img_buf)
        #imageArray.append(im)

        plt.show()
    #return imageArray

In [ ]:
plotOverIterations(1,Data_test)

In [ ]:
imagesToSave = 10
rnd = np.arange(imagesToSave,dtype='int32')
np.random.shuffle(rnd)
for i in range(10):
    im_array = plotOverIterations(rnd[i],Data)
    im = im_array[0]
    im_array.pop(0)
    im.save("out{}.gif".format(i),save_all=True,append_images = im_array,optimize=False,loop=0)
    im.close()

<h1>Thoughts</h1>

M6: Looking at the model at work shows progress when loads are grouped up. Failures seem to occur with spaced load conditions and low volfrac.

For the Unet, in order to be able to capture multiple resolutions we need to allow the downsampling to resolve issues with resolutions that are not multiples of 2.

This can be solved with the following setup
Take the resolution when a down sample occurs. Take the ceil of the down sample(5/2 = 3). Pass this to the next layer and continue untill upscaleing. When we upscale there is a possibility of the tensors that must be concatenated not being the same resolution. We take the resolution with the higher dimensions(this will always be the upsampled part, not the skip connection) and crop it by one along the nessesary dimension.

Some times the cropping will be nessesary sometimes it will not be nessesary but it will always be a cropping by 1.

In [ ]:

def buildModel_m9_dropout(x_inputShape = (101,51,1),LoadConditionsImage = (101,51,6),activation='relu'):
    partInput = tf.keras.layers.Input(shape=x_inputShape,name="x")
    loadsInput = tf.keras.layers.Input(shape=LoadConditionsImage,name="loadConditions")

    concatenatedStartLayer = tf.keras.layers.Concatenate()([partInput,loadsInput])

    #First Convolution Layer
    conv_128_64 = tf.keras.layers.Conv2D(filters= 16, kernel_size=(3,3),padding='same',activation=activation)(concatenatedStartLayer)
    conv_128_64 = tf.keras.layers.Conv2D(filters= 16, kernel_size=(3,3),padding='same',activation=activation)(conv_128_64)
    conv_64_32 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding='same')(conv_128_64)
    conv_64_32 = tf.keras.layers.GaussianNoise(stddev=0.1)(conv_64_32)
    conv_64_32 = tf.keras.layers.Dropout(rate=0.1)(conv_64_32)


    #Second convolution Layer
    conv_64_32 = tf.keras.layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(conv_64_32)
    conv_64_32 = tf.keras.layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(conv_64_32)
    conv_32_16 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding='same')(conv_64_32)
    conv_32_16 = tf.keras.layers.GaussianNoise(stddev=0.1)(conv_32_16)
    conv_32_16 = tf.keras.layers.Dropout(rate=0.1)(conv_32_16)

    conv_32_16 = tf.keras.layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(conv_32_16)
    conv_32_16 = tf.keras.layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(conv_32_16)
    conv_16_8 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding='same')(conv_32_16)
    conv_16_8 = tf.keras.layers.GaussianNoise(stddev=0.1)(conv_16_8)
    conv_16_8 = tf.keras.layers.Dropout(rate=0.1)(conv_16_8)

    conv_16_8 = tf.keras.layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(conv_16_8)
    conv_16_8 = tf.keras.layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(conv_16_8)

    #upscaleLayer
    #upscaling is performed by convolution transpose where stride=2 < kernalsize
    convUpscale_32_16 = tf.keras.layers.Conv2DTranspose(filters= 64, kernel_size=(5,5),strides=2,padding='same',activation=activation)(conv_16_8)
    convUpscale_32_16 = tf.keras.layers.GaussianNoise(stddev=0.1)(convUpscale_32_16)
    convUpscale_32_16 = ConcatAndCrop(convUpscale_32_16.shape,conv_32_16.shape)(convUpscale_32_16,conv_32_16)
    convUpscale_32_16 = tf.keras.layers.Conv2D(filters = 64, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_32_16)
    convUpscale_32_16 = tf.keras.layers.Conv2D(filters = 64, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_32_16)
    convUpscale_32_16 = tf.keras.layers.Dropout(rate=0.1)(convUpscale_32_16)

    convUpscale_64_32 = tf.keras.layers.Conv2DTranspose(filters= 32, kernel_size=(5,5),strides=2,padding='same',activation=activation)(convUpscale_32_16)
    convUpscale_64_32 = tf.keras.layers.GaussianNoise(stddev=0.1)(convUpscale_64_32)
    convUpscale_64_32 = ConcatAndCrop(convUpscale_64_32.shape,conv_64_32.shape)(convUpscale_64_32,conv_64_32)
    convUpscale_64_32 = tf.keras.layers.Conv2D(filters = 32, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_64_32)
    convUpscale_64_32 = tf.keras.layers.Conv2D(filters = 32, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_64_32)
    convUpscale_64_32 = tf.keras.layers.Dropout(rate=0.1)(convUpscale_64_32)

    convUpscale_128_64 = tf.keras.layers.Conv2DTranspose(filters= 64, kernel_size=(5,5),strides=2,padding='same',activation=activation)(convUpscale_64_32)
    convUpscale_64_32 = tf.keras.layers.GaussianNoise(stddev=0.1)(convUpscale_64_32)
    convUpscale_128_64 = ConcatAndCrop(convUpscale_128_64.shape,conv_128_64.shape)(convUpscale_128_64,conv_128_64)
    convUpscale_128_64 = tf.keras.layers.Conv2D(filters = 16, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_128_64)
    convUpscale_128_64 = tf.keras.layers.Conv2D(filters = 16, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_128_64)

    output_part = tf.keras.layers.Conv2D(filters= 1, kernel_size=(1,1),padding='same',activation='hard_sigmoid', name="x_out")(convUpscale_128_64)
    """
    The hard sigmoid activation, defined as:
        if x < -2.5: return 0
        if x > 2.5: return 1
        if -2.5 <= x <= 2.5: return 0.2 * x + 0.5
    """

    return tf.keras.Model(inputs= [partInput,loadsInput],outputs=[output_part])#,finishedOutput])

class Model_m9_dropout(tf.keras.Model):
    def __init__(self,nelx:int=101,nely:int=51):

        super(Model_m9_dropout, self).__init__()

        self.model = buildModel_m9_dropout((nelx,nely,1),(nelx,nely,6))

        

    def call(self,data,training = False):
        #part = data['x']
        #forces = data['forces']
        #supports = data['supports']
        #filled = data['filled']
        #print(1)
        if(training):
            
            out1 = self.model(data)
            data['x'] = out1
            
            #print(2)
            out2 = self.model(data)
            data['x'] = out2

            out3 = self.model(data)
            data['x'] = out3
        
            out4 = self.model(data)
            data['x'] = out4

            out5 = self.model(data)
            #print(6)
            return out1,out2,out3,out4,out5
        else:
            return self.model(data)



In [ ]:
m2 = Model_m9_dropout()
m2.model.summary()

In [ ]:
modelName = "Model_m9"

modelPath = os.path.join(os.getcwd(),'ModelSave',modelName)
m2.load_weights(os.path.join(modelPath,modelName))

In [ ]:
m2.set_weights()

In [ ]:
for i,layer in enumerate(m2.model.layers):
#layer = model.model.layers[1].output
    name = layer.name
    #if('conv2d' in name):
    print(i,name)
print(len(model.model.layers))

In [ ]:
for i,layer in enumerate(model.model.layers):
#layer = model.model.layers[1].output
    name = layer.name
    #if('conv2d' in name):
    print(i,name)
print(len(model.model.layers))

In [ ]:
matchups = [[11,4],[14,7],[15,8],[18,11],[19,12],[22,15],[23,16],[24,17],[27,20],[28,21],[29,22],[32,25],[33,26],[34,27],[36,29],[37,30],[39,31]]

In [ ]:
for i,m2_layer in enumerate(m2.model.layers):
    for j,m1_layer in enumerate(model.model.layers):
        for iMatch,jMatch in matchups:
            if(i==iMatch and j == jMatch):
                print(f"Getting weights from {i}:{m2_layer.name} and putting them in {j}:{m1_layer.name}")
                m1_layer.set_weights(m2_layer.get_weights())

In [ ]:
modelName = "Model_m9"

modelPath = os.path.join(os.getcwd(),'ModelSave',modelName)
model.save_weights(modelPath)